In [1]:
!pip install catboost

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings(action='ignore')
import catboost
from catboost import *
import sklearn
from sklearn.preprocessing import LabelEncoder
import sys

from itertools import product
from sklearn.tree import DecisionTreeClassifier
from sklearn import metrics
import copy

In [3]:
train_df = pd.read_csv('/content/drive/MyDrive/LGaimers2/input/train.csv')
test = pd.read_csv('/content/drive/MyDrive/LGaimers2/input/test.csv')
submit = pd.read_csv('/content/drive/MyDrive/LGaimers2/input/sample_submission.csv')

In [4]:
def del_columns(train, test):
    col_list = train.columns
    nan_list = []
    nan_cnt = []
    nan_col = []
    full_list = []
    for col in col_list:
        if train[col].isnull().sum() == 0 :
            full_list.append(col)
            continue
        nan_list.append([col, train[col].isnull().sum()])
        nan_cnt.append(train[col].isnull().sum())
        nan_col.append(col)

    del_col = []
    for col in nan_list :
        if col[1] == len(train) :
            del_col.append(col[0])
    train = train.drop(columns=del_col)
    test = test.drop(columns=del_col)

    del_col = []
    col_list = train.describe().columns
    for col in col_list :
        if col == 'Y_Class':
            continue
        if col == 'Y_Quality':
            continue
        if col == 'LINE':
            continue
        if col == 'PRODUCT_CODE':
            continue
        if train[col].nunique()==1 :
            del_col.append(col)
    train = train.drop(columns=del_col)
    test = test.drop(columns=del_col)
    
    return train,test

def make_train_test_dataset(train,test):
    
    '''
    트레인데이터, 학습데이터 셋 만들기
    '''
    
    train_x = train.drop(columns=['PRODUCT_ID','PRODUCT_CODE','Y_Class','Y_Quality'])
    test_x = test.drop(columns=['PRODUCT_ID','PRODUCT_CODE'])
    train_y = train['Y_Quality']
    train_w = train[['Y_Class']]
    return train_x, test_x, train_y, train_w


def fillna(train,test,value):
    train = train.fillna(value)
    test = test.fillna(value)
    return train,test

def labelencoder(train,test,col_list):
    
    qual_col = col_list
    for i in qual_col:
        le = LabelEncoder()
        le = le.fit(train[i])
        train[i] = le.transform(train[i])

        for label in np.unique(test[i]): 
            if label not in le.classes_: 
                le.classes_ = np.append(le.classes_, label)
        test[i] = le.transform(test[i]) 
    return train,test

In [18]:
#train, test = del_columns(train,test)
train, test = fillna(train_df,test,-1)
#train_x, test_x, train_y, train_w = make_train_test_dataset(train,test)
#train_x, test_x = labelencoder(train_x,test_x,['LINE'])
train['PRODUCT_CODE'] = train['PRODUCT_CODE'].astype('category')
train['LINE'] = train['LINE'].astype('category')

test['PRODUCT_CODE'] = test['PRODUCT_CODE'].astype('category')
test['LINE'] = test['LINE'].astype('category')

In [19]:
rows_with_all_minus_one_tr = []

# 2880번째 열부터 마지막 열까지 각 행에 대해 반복
for index, row in train.iloc[:, 2880:].iterrows():
    # 행의 모든 값이 -1인지 확인
    if all(value == -1 for value in row):
        rows_with_all_minus_one_tr.append(index)

# 결과를 저장할 빈 데이터프레임 생성
result_data_tr = train.iloc[rows_with_all_minus_one_tr]

rows_with_all_minus_one_te = []

# 2880번째 열부터 마지막 열까지 각 행에 대해 반복
for index, row in test.iloc[:, 2880:].iterrows():
    # 행의 모든 값이 -1인지 확인
    if all(value == -1 for value in row):
        rows_with_all_minus_one_te.append(index)

# 결과를 저장할 빈 데이터프레임 생성
result_data_te = test.iloc[rows_with_all_minus_one_te]

train_bf = train.iloc[rows_with_all_minus_one_tr]
test_bf = test.iloc[rows_with_all_minus_one_te]
submit_bf = submit.iloc[rows_with_all_minus_one_te]

train_af= train.drop(result_data_tr.index)
test_af= test.drop(result_data_te.index)
submit_af= submit.drop(result_data_te.index)

train_bf = train_bf.reset_index(drop=True)
test_bf = test_bf.reset_index(drop=True)

train_af = train_af.reset_index(drop=True)
test_af = test_af.reset_index(drop=True)

In [20]:
def get_best_threshold_spliter(y_pred, y_true_cls, model=None, training=True):
    if training:
        search_space = [[2, 4, 6], [1, 3, 5, 7, 9]]
        best_score = -np.inf
        output_pred = []
        model = None
        for depth, min_samples in product(*search_space): 
            model = DecisionTreeClassifier(
                criterion="gini", max_features=1.0,
                max_depth=depth, min_samples_leaf=min_samples, random_state=42
            )
            model.fit(y_pred, y_true_cls)
            y_pred_cls = model.predict(y_pred)
            score = metrics.f1_score(y_true_cls, y_pred_cls, average="macro")
            if best_score < score:
                best_score = score
                print(f"Best score : {best_score}")
                output_pred = y_pred_cls.copy()
                model = copy.deepcopy(model)
        return model, output_pred
    else:
        output_pred = model.predict(y_pred)
        return output_pred

In [21]:
train_bf, test_bf = del_columns(train_bf,test_bf)
train_af, test_af = del_columns(train_af,test_af)

In [22]:
train_x_bf, test_x_bf, train_y_bf, train_w_bf = make_train_test_dataset(train_bf,test_bf)
train_x_af, test_x_af, train_y_af, train_w_af = make_train_test_dataset(train_af,test_af)

In [23]:
line_bf = train_x_bf['LINE']
line_af = train_x_af['LINE']

except_line_bf = train_x_bf.drop(columns='LINE')
except_line_af = train_x_af.drop(columns='LINE')

In [24]:
import scipy.stats as stats

# 자기 자신 포함 corr 높은 변수명 저장
def correlation(dataframe, standard):
    doubt = []
    for i in dataframe.columns:
        corr , p_val = stats.pearsonr(dataframe[standard], dataframe[i])
        if abs(corr) >= 1 and p_val < 0.05:
            print('standard : ', standard ,'    ', 'i : ', i, ' ', stats.pearsonr(dataframe[standard], dataframe[i]))
            doubt.append(i)
    return doubt

In [25]:
def make_preprocess_dataset(train_df, test_df, standard):
    doubt = correlation(train_df, standard)

    counts=[]
    del_cols = []
    for i in doubt:
        counts.append(len(train_df.loc[train_df[i] == -1]))
    standard = doubt[counts.index(min(counts))] #corr 높은 애 중에 결측치 값이 가장 적은 것 기준.

    del_cols = [i for i in doubt if i != standard]

    print(doubt, del_cols)

    train_df = train_df.drop(columns=del_cols)
    test_df = test_df.drop(columns=del_cols)

    return train_df, test_df

In [26]:
tmp = except_line_bf.copy()
tmp_test = test_x_bf.copy()

tmp2 = except_line_af.copy()
tmp2_test = test_x_af.copy()

In [ ]:
for i in except_line_bf:
    try:
        tmp, tmp_test = make_preprocess_dataset(tmp, tmp_test, i)
    except:
        continue   
tmp

standard :  X_1      i :  X_1   PearsonRResult(statistic=1.0, pvalue=0.0)
['X_1'] []
standard :  X_3      i :  X_3   PearsonRResult(statistic=1.0, pvalue=0.0)
standard :  X_3      i :  X_6   PearsonRResult(statistic=1.0, pvalue=0.0)
standard :  X_3      i :  X_10   PearsonRResult(statistic=1.0, pvalue=0.0)
standard :  X_3      i :  X_14   PearsonRResult(statistic=1.0, pvalue=0.0)
standard :  X_3      i :  X_19   PearsonRResult(statistic=1.0, pvalue=0.0)
standard :  X_3      i :  X_23   PearsonRResult(statistic=1.0, pvalue=0.0)
standard :  X_3      i :  X_25   PearsonRResult(statistic=1.0, pvalue=0.0)
standard :  X_3      i :  X_29   PearsonRResult(statistic=1.0, pvalue=0.0)
standard :  X_3      i :  X_30   PearsonRResult(statistic=1.0, pvalue=0.0)
standard :  X_3      i :  X_31   PearsonRResult(statistic=1.0, pvalue=0.0)
standard :  X_3      i :  X_32   PearsonRResult(statistic=1.0, pvalue=0.0)
standard :  X_3      i :  X_33   PearsonRResult(statistic=1.0, pvalue=0.0)
standard :  X_3  

In [ ]:
for i in except_line_af:
    try:
        tmp2, tmp2_test = make_preprocess_dataset(tmp2, tmp2_test, i)
    except:
        continue   
tmp2

In [ ]:
tmp.insert(0, 'LINE', line_bf.values )
tmp2.insert(0, 'LINE', line_af.values )

#Train before

In [ ]:
bf_stats = pd.DataFrame({"qual": train_y_bf, "cls": train_w_bf["Y_Class"]}).groupby("cls")["qual"].describe()
af_stats = pd.DataFrame({"qual": train_y_af, "cls": train_w_af["Y_Class"]}).groupby("cls")["qual"].describe()

In [ ]:
clf = catboost.CatBoostRegressor(
    learning_rate=0.05,
    iterations=1000,
    depth=6,
    l2_leaf_reg=5,
    border_count=254,
    cat_features=['LINE'],
    random_seed=313,
    verbose=False,
)

In [ ]:
train_x_bf = tmp.copy()
test_x_bf = tmp_test.copy()

train_x_af = tmp2.copy()
test_x_af = tmp2_test.copy()

In [ ]:
clf.fit(train_x_bf, train_y_bf)
train_pred_bf = clf.predict(train_x_bf)
pred_bf = clf.predict(test_x_bf)

In [ ]:
reg_prob = np.abs(np.stack([
    train_pred_bf - bf_stats.loc[0, "mean"],
    train_pred_bf - bf_stats.loc[1, "mean"],
    train_pred_bf - bf_stats.loc[2, "mean"],
], axis=1))
model_threshold, train_pred_bf = get_best_threshold_spliter(reg_prob, train_w_bf["Y_Class"].values, training=True)

reg_prob = np.abs(np.stack([
    pred_bf - bf_stats.loc[0, "mean"],
    pred_bf - bf_stats.loc[1, "mean"],
    pred_bf - bf_stats.loc[2, "mean"],
], axis=1))
pred_bf = get_best_threshold_spliter(reg_prob, y_true_cls=None, model=model_threshold, training=False)

#Train after

In [ ]:
#clf = catboost.CatBoostClassifier(verbose=0,cat_features=['LINE'])
clf.fit(train_x_af, train_y_af)
train_pred_af = clf.predict(train_x_af)
pred_af = clf.predict(test_x_af)

In [ ]:
reg_prob = np.abs(np.stack([
    train_pred_af - af_stats.loc[0, "mean"],
    train_pred_af - af_stats.loc[1, "mean"],
    train_pred_af - af_stats.loc[2, "mean"],
], axis=1))
model_threshold, train_pred_af = get_best_threshold_spliter(reg_prob, train_w_af["Y_Class"].values, training=True)

reg_prob = np.abs(np.stack([
    pred_af - af_stats.loc[0, "mean"],
    pred_af - af_stats.loc[1, "mean"],
    pred_af - af_stats.loc[2, "mean"],
], axis=1))
pred_af = get_best_threshold_spliter(reg_prob, y_true_cls=None, model=model_threshold, training=False)

Submission

In [ ]:
submit_bf['Y_Class'] = pred_bf
submit_af['Y_Class'] = pred_af

In [ ]:
submit_fin = pd.concat([submit_bf, submit_af],axis=0)
submit_fin = submit_fin.sort_values('PRODUCT_ID')
submit_fin.to_csv('reg_1_changeseed_313_dt.csv', index=False)

In [ ]:
submit_fin['Y_Class'].value_counts()

In [ ]:
submit_fin